In [ ]:
import pandas as pd
import yfinance as yf
import ta
import plotly.graph_objs as go


In [ ]:
sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]

sp500["Symbol"] = sp500["Symbol"].str.replace('.', "-")

In [ ]:
sp500

In [ ]:
symbol_list = sp500["Symbol"].unique().tolist()

In [ ]:
end_date = "2024-02-25"

start_date = "2015-01-30"

df = yf.download(tickers=symbol_list,
                 start=start_date,
                 end=end_date).stack()

In [ ]:
df.index.names = ['date', 'ticker']
df.columns = df.columns.str.lower()  
df

In [ ]:
df.to_csv("stock.csv")

In [ ]:
stocks = pd.read_csv("Data/stock.csv")
stocks.tail()

In [ ]:
stocks

In [ ]:
tickers = stocks['ticker'].unique().tolist()
tickers = sorted(tickers)

In [ ]:


# Set the index to be a MultiIndex with 'ticker' and 'date'
multi_index_df = stocks.set_index(['ticker', 'date'])

multi_index_df.sort_index(inplace=True)

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)  
pd.set_option('display.max_rows', 100) 


multi_index_df


In [ ]:
tickers

In [ ]:
#Price
price = []
price.append((multi_index_df['high'] + multi_index_df['low'] + multi_index_df['adj close']) / 3)


#VWAP
pv = multi_index_df['volume'] * price[0]

cumu_pv = pv.cumsum()
cumu_vol = multi_index_df['volume'].cumsum()

multi_index_df['vwap'] = cumu_pv / cumu_vol
multi_index_df

In [ ]:
multi_index_df

In [ ]:
df = multi_index_df.copy()

In [ ]:

processed_df = pd.DataFrame()

for tick in tickers:
    ticker_data = df.loc[tick].copy()
    
    # Calculate Relative Strength Index
    ticker_data['rsi'] = ta.momentum.rsi(ticker_data['close'], window=14)

    # Calculate Bollinger Bands
    bollinger = ta.volatility.BollingerBands(close=ticker_data['close'], window=20, window_dev=2)
    ticker_data['bb_low'] = bollinger.bollinger_lband()
    ticker_data['bb_mid'] = bollinger.bollinger_mavg()
    ticker_data['bb_high'] = bollinger.bollinger_hband()

    # Calculate Average True Range
    ticker_data['atr'] = ta.volatility.average_true_range(ticker_data['high'], ticker_data['low'], ticker_data['close'], window=14)

    # Calculate Moving Average Convergence Divergence
    macd = ta.trend.MACD(ticker_data['close'])
    ticker_data['macd'] = macd.macd()
    ticker_data['macd_signal'] = macd.macd_signal()
    
    # Calculate Moving Averages
    ticker_data['short_ma'] = ta.trend.sma_indicator(ticker_data['close'], window=20)  
    ticker_data['long_ma'] = ta.trend.sma_indicator(ticker_data['close'], window=50)
    
    ticker_data['return'] = ticker_data['adj close'].pct_change()
    
    ticker_data['rolling_mean'] = ticker_data['adj close'].rolling(window=20).mean()
    ticker_data['rolling_std'] = ticker_data['adj close'].rolling(window=20).std()
    
    ticker_data['lag1'] = ticker_data['adj close'].shift(1)
    ticker_data['lag2'] = ticker_data['adj close'].shift(2)
    ticker_data['lag3'] = ticker_data['adj close'].shift(3)
    
    ticker_data['return'] = ticker_data['adj close'].pct_change()

    
    ticker_data['target'] = (ticker_data['close'].shift(-1) > ticker_data['close']).astype(int)
    # Remove NaN values
    ticker_data = ticker_data.dropna(subset=['rsi', 'bb_low', 'bb_mid', 'bb_high', 'atr', 'macd', 'macd_signal', 'short_ma', 'long_ma', 'lag1', 'lag2', 'lag3', 'rolling_mean', 'rolling_std', 'return'])

    ticker_data['ticker'] = tick
    ticker_data = ticker_data.set_index('ticker', append=True)
    ticker_data = ticker_data.reorder_levels(['ticker', 'date'])


    processed_df = pd.concat([processed_df, ticker_data])
    
    



In [ ]:
ticker = ['AAPL', 'A', 'AAL']

def plot_market(data, ticker:list, market:str):
    fig = go.Figure()
    for tick in ticker:
        if tick not in tickers:
            return f'{tick} not valid ticker'
    if market == 'volume':
        yaxis = 'Volume' 
    elif market in data.columns:
        yaxis = f'{market.capitalize()} Prince'
    else:
        return 'Enter valid Market'
        
    for tick in ticker:
        fig.add_trace(go.Scatter(
            x=data.loc[tick].index,
            y=data.loc[tick][market],
            mode='lines',
            name=tick
        ))
    fig.update_layout(
        title=f'Stock {market} for Multiple Ticker/s',
        xaxis_title='Date',            
        yaxis_title=yaxis,
        legend_title='Ticker'
    )
    fig.show()
    

plot_market(processed_df, ticker=ticker, market='short_ma' )


In [ ]:
df = processed_df.copy()


In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go



app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='stock-selector',
        options=[{'label': ticker, 'value': ticker} for ticker in tickers],
        value=df.index.get_level_values(0).unique()[0]
    ),
    dcc.Graph(id='price-chart')
])

@app.callback(
    Output('price-chart', 'figure'),
    [Input('stock-selector', 'value')]
)
def update_chart(selected_stock):
    ticker_df = df.loc[selected_stock]
    figure = {
        'data': [
            go.Scatter(x=ticker_df.index, y=ticker_df['close'], mode='lines', name='Close Price'),
            go.Scatter(x=ticker_df.index, y=ticker_df['short_ma'], mode='lines', name='Short MA'),
            go.Scatter(x=ticker_df.index, y=ticker_df['long_ma'], mode='lines', name='Long MA')
        ],
        'layout': go.Layout(
            title=f'Stock Prices for {selected_stock}',
            xaxis={'title': 'Date'},
            yaxis={'title': 'Price'}
        )
    }
    return figure

if __name__ == '__main__':
    app.run_server(debug=True)



In [ ]:
# import pandas as pd
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error

# # Example tickers list

# df.index = df.index.set_levels([df.index.levels[0], pd.to_datetime(df.index.levels[1])])

# # Define the cutoff date
# cutoff_date = pd.to_datetime('2023-01-01')

# models = {}
# mean_sqr_error = {}
# predictions_dict = {}
# actuals_dict = {}
# dates_dict = {}

# count = 0
# for tick in tickers:
#     stock_data = df.loc[tick].copy() 

    
#     # Prepare features and target
#     X = stock_data[['lag1', 'lag2', 'lag3', 'rolling_mean', 'rolling_std', 'volume', 'rsi', 'bb_low', 'bb_mid', 'bb_high', 'atr', 'macd', 'macd_signal', 'return']]
#     y = stock_data['adj close']

#     # Split into training and testing sets
#     train_data = stock_data.loc[stock_data.index < stock_data.index[round(len(stock_data.index) * .872)]]
#     test_data = stock_data.loc[stock_data.index >= stock_data.index[round(len(stock_data.index) * .872)]]
    
#     X_train = train_data.drop(columns=['adj close'])
#     y_train = train_data['adj close']
#     X_test = test_data.drop(columns=['adj close'])
#     y_test = test_data['adj close']
    
#     # Model Training
#     model = RandomForestRegressor(n_estimators=100, random_state=42)
    
#     print(f'Shape: {X_train.shape}')
#     model.fit(X_train, y_train)
    
#     # Save the model
#     models[tick] = model
    
#     # Model Prediction and Evaluation
#     predictions = model.predict(X_test)
    
#     predictions_dict[tick] = predictions
#     actuals_dict[tick] = y_test.values
#     dates_dict[tick] = test_data.index
    
#     mean_sqr_error[tick] = mean_squared_error(y_test, predictions)
#     mse = mean_sqr_error[tick]
    
#     count +=1 
#     print(f'{count}: {tick} - Mean Squared Error: {mse}')
    



In [ ]:
# import pickle as pk

# with open('MSE.pkl', 'wb') as f:
#     pk.dump(mean_sqr_error, f)
    
# with open('dates.pkl', 'wb') as f:
#     pk.dump(dates_dict, f)
    
# with open('actuals.pkl', 'wb') as f:
#     pk.dump(actuals_dict, f)
    
# with open('predictions.pkl', 'wb') as f:
#     pk.dump(predictions_dict, f)

In [ ]:
import pickle as pk

with open('Data/MSE.pkl', 'rb') as f:
    mean_sqr_error = pk.load(f)
with open ('Data/dates.pkl', 'rb') as f:
    dates_dict = pk.load(f)
with open ('Data/actuals.pkl', 'rb') as f:
    actuals_dict = pk.load(f)
with open ('Data/predictions.pkl', 'rb') as f:
    predictions_dict = pk.load(f)

In [ ]:
with open('Data/stock_model_2.pkl', 'rb') as f:
    models = pk.load(f)

In [ ]:
import plotly.io as pio

figures_json = {}
for ticker in tickers:
    predictions = predictions_dict[ticker]
    actuals = actuals_dict[ticker]
    dates = dates_dict[ticker]
    
    # Retrieve the Mean Squared Error from the dictionary
    mse = mean_sqr_error[ticker]
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=dates,
        y=actuals,
        mode='lines',
        name='Actual',
        line=dict(color='blue')
    ))

    fig.add_trace(go.Scatter(
        x=dates,
        y=predictions,
        mode='lines',
        name=f'Predicted (MSE: {mse:.2f})',
        line=dict(color='red')
    ))

    fig.update_layout(
        title=f'Actual vs Predicted values for {ticker}',
        xaxis_title='Date',
        yaxis_title='Price',
        legend_title='Legend'
    )
    
    figures_json[ticker] = pio.to_json(fig)

html_content = '''
<!DOCTYPE html>
<html>
<head>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
</head>
<body>
    <h1>Stock Price Predictions</h1>
    <label for="ticker-select">Choose a ticker:</label>
    <select id="ticker-select" onchange="updatePlot()">
        {options}
    </select>
    <div id="plot"></div>
    <script>
        var figures = {figures};
        
        function updatePlot() {{
            var ticker = document.getElementById('ticker-select').value;
            var graphDiv = document.getElementById('plot');
            Plotly.react(graphDiv, JSON.parse(figures[ticker]).data, JSON.parse(figures[ticker]).layout);
        }}
        
        // Initial plot
        document.addEventListener('DOMContentLoaded', function() {{
            updatePlot();
        }});
    </script>
</body>
</html>
'''

options = ''.join([f'<option value="{ticker}">{ticker}</option>' for ticker in tickers])
html_content = html_content.format(options=options, figures=figures_json)

with open('stock_predictions.html', 'w') as f:
    f.write(html_content)

print("HTML file with dropdown has been generated.")